In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from IPython.display import JSON

**Remove API Key Before Publishing to GitHub**

In [ ]:
# Remove API key before pushing to GitHub
api_key = "API_KEY"

Youtube Channel

In [ ]:
#Can place multiple channel ids in the list, 0 = Open AI, 1 = Kurzgesagt
channel_ids = ["Open AI", "Kurzgesagt"]

In [ ]:
import httplib2
from googleapiclient.discovery import build

Describe API

In [ ]:
  api_service_name = "youtube"
  api_version = "v3"

   
youtube = build(
    api_service_name, api_version, developerKey=api_key)
request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=",".join(channel_ids)  # concatenates channel ids with commas
)
response = request.execute()

In [ ]:
JSON(response)

Getting Channel Stats

In [ ]:
def get_channel_stats(youtube, channel_ids):

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)  # concatenates channel ids with commas
    )
    response = request.execute()
    all_data = []
    # Loop through each channel and get the stats
    for item in response['items']:
        data = {
            "channel Name": item['snippet']['title'],
            "channelid": item['id'],
            "channel description": item['snippet']['description'],
            "channel subscribers": item['statistics']['subscriberCount'],
            "channel video count": item['statistics']['videoCount'],
            "channel view count": item['statistics']['viewCount'],
            "uploads id": item['contentDetails']['relatedPlaylists']['uploads'] # add a check for the key
        }
        all_data.append(data)
    
    return pd.DataFrame(all_data)

In [ ]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [ ]:
channel_stats

Getting Video ID's for Kurzgesagt

In [ ]:
#Uploads id from channel stats
playlist_id = "UUsXVk37bltHxD1rDPwtNM8Q"

def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId = playlist_id,
        maxResults = 50,

    )
    response = request.execute()
   
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId = playlist_id,
        maxResults = 50,
        pageToken = next_page_token

        )
        response = request.execute()
    
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    return video_ids

In [ ]:
video_ids = get_video_ids(youtube, playlist_id)

In [ ]:
len(video_ids)

In [ ]:
video_ids

Getting the Video Stats for the all of the Video's we pulled from Kurzgesagt

In [ ]:
def get_video_details(youtube, video_ids):
    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        ) 
        response = request.execute()
            
        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                            'statistics': ['viewCount', 'likeCount', 'favoriteCount', 'commentCount'],
                            'contentDetails': ['duration', 'definition', 'caption']}
            video_info = {}
            video_info['video_id'] = video['id']

            for s in stats_to_keep:
                for v in stats_to_keep[s]:
                    try:
                        video_info[v] = video[s][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)


In [ ]:
JSON(response)

Video details

In [ ]:
video_df = get_video_details(youtube, video_ids)

In [ ]:
video_df

In [ ]:
selected_video_ids = video_ids[0:200]
video_details_df = get_video_details(youtube, selected_video_ids)

In [ ]:
video_details_df

Use code below to clean the comments before we save them, Skip this and remove the function here "comments_in_video = [
                clean_text()" to keep the raw comments  

Clean text two

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
import re
import emoji

Origional comments request, get's one page per video, aproximitly 50 comment's per page. We Can Skip this def and use the 3rd one below.
___

Can be deleted

In [ ]:
from googleapiclient.errors import HttpError
def get_comments(youtube, video_ids):
    all_comments = []

# We'll include try a and except to continue if the comments are disabled for a video

    for video_id in video_ids:
        try:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
                order="relevance",
                maxResults = 50
            
            )
            response = request.execute()
        #Can replace original_comments with comments_in_video and remove "comments_in_video = preprocess_comments(original_comments)" to go back to original comments
            comments_in_video = [
                (comment['snippet']['topLevelComment']['snippet']['textOriginal'])
                for comment in response['items']
            ]
             # Preprocess the comments, remove to go back to original comments
            #comments_in_video = preprocess_comments(original_comments)

            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
        except HttpError as e:
            print(f"HttpError for video_id {video_id}: {e}")
            continue

    return pd.DataFrame(all_comments)

In [ ]:
comments_df = get_comments(youtube, video_ids)

(Def 2) Get's 2 pages of comments approximently 50 comments per page.
Can be deleted.
___

In [ ]:
from googleapiclient.errors import HttpError

def get_comments(youtube, video_ids):
    all_comments = []

    for video_id in video_ids:
        try:
            nextPageToken = None  # Initialize the nextPageToken for each video
            page_count = 0  # Initialize a counter for the number of pages processed

            while page_count < 2:  # Fetch two pages then stop
                request = youtube.commentThreads().list(
                    part="snippet,replies",
                    videoId=video_id,
                    order="relevance",
                    maxResults=50,
                    pageToken=nextPageToken
                )
                response = request.execute()

                # Extract comments
                comments_in_video = [
                    comment['snippet']['topLevelComment']['snippet']['textOriginal']
                    for comment in response['items']
                ]

                # Add the comments to your list
                all_comments.append({
                    'video_id': video_id,
                    'comments': comments_in_video
                })

                # Increment the page counter
                page_count += 1

                # Check if there is a nextPageToken, and update nextPageToken
                nextPageToken = response.get('nextPageToken')
                if not nextPageToken:
                    break  # Exit the loop if no more pages

        except HttpError as e:
            print(f"HttpError for video_id {video_id}: {e}")
            continue

    # Combine all comments into a DataFrame
    return pd.DataFrame(all_comments)


Removing emojis, currently commented out to keep the origional comments. You can delete this if you want.

In [ ]:
#import re

#def remove_emojis(text):
   # """Remove emojis from the text using regular expression."""
    #emoji_pattern = re.compile("["
                          # u"\U0001F600-\U0001F64F"  # emoticons
                          # u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                          # u"\U0001F680-\U0001F6FF"  # transport & map symbols
                          # u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          # u"\U00002702-\U000027B0"
                           #u"\U000024C2-\U0001F251"
                           #u"\U0001F900-\U0001FAFF"  # Supplemental Symbols and Pictographs + Emoticons
                           #u"\U0001F700-\U0001F77F"  # Alchemical Symbols
                           #u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           #u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           #u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           #u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           #u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           #u"\U0001FA70-\U0001FAFF" # Symbols and Pictographs Extended-A
                           #u"\U0001FB00-\U0001FBFF" # Symbols for Legacy Computing
                           #"]+", flags=re.UNICODE)
    #return emoji_pattern.sub(r'', text)


(Def 3) Use this def to fetch comments, the ones above can be ignored

In [ ]:
from googleapiclient.errors import HttpError

def get_comments(youtube, video_ids):
    all_comments = []

    for video_id in video_ids:
        try:
            nextPageToken = None  # Initialize the nextPageToken for each video
            page_count = 0  # Initialize a counter for the number of pages processed

            while page_count < 2:  # Fetch two pages then stop
                request = youtube.commentThreads().list(
                    part="snippet,replies",
                    videoId=video_id,
                    order="relevance",
                    maxResults=50, # Getting about 100 comments per video since we are fetching 2 pages 
                    pageToken=nextPageToken
                )
                response = request.execute()

                # Extract comments
                comments_in_page = [
                    comment['snippet']['topLevelComment']['snippet']['textOriginal'] #to go bact to original comments use "comment['snippet']['topLevelComment']['snippet']['textOriginal']" without remove_emojis
                    for comment in response['items']
                ]

                # Store each page of comments separately
                all_comments.append({
                    'video_id': video_id,
                    'comments': comments_in_page
                })

                # Increment the page counter and update nextPageToken
                page_count += 1
                nextPageToken = response.get('nextPageToken')
                if not nextPageToken:
                    break

        except HttpError as e:
            print(f"HttpError for video_id {video_id}: {e}")
            continue

    # Combine all comments into a DataFrame
    return pd.DataFrame(all_comments)


In [ ]:
# Run this to get the comments
comments_df = get_comments(youtube, video_ids)

In [ ]:
# Grouping comments by the video ID
total_comments_df = comments_df.groupby('video_id')['comments'].agg(list).reset_index()

In [ ]:
import pandas as pd

# Assuming comments_df is your DataFrame with columns 'video_id' and 'comments'
# Where 'comments' are lists of comments

def concatenate_comment_lists(series):
    # This function concatenates lists of comments into a single list
    concatenated_list = []
    for comment_list in series:
        concatenated_list.append(comment_list)
    return concatenated_list

# Group by 'video_id' and concatenate the comments
grouped_comments_df = comments_df.groupby('video_id')['comments'].apply(concatenate_comment_lists).reset_index()

# Now, grouped_comments_df has each 'video_id' associated with a concatenated list of comments
print(grouped_comments_df.head())


In [ ]:
comments_df

Data Pre-processing

In [ ]:
video_details_df.isnull().any()

In [ ]:
numeric_cols = ['viewCount', 'likeCount', 'favoriteCount', 'commentCount']
video_details_df[numeric_cols] = video_details_df[numeric_cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [ ]:
from dateutil.parser import parse

Use code below to get publish day and time. Code above can be deleted.

In [ ]:

# Published day of the Week

video_details_df['publishedAt'] = video_details_df['publishedAt'].apply(lambda x: parse(x))
video_details_df['publishDayName'] = video_details_df['publishedAt'].apply(lambda x: x.strftime('%A'))

Below is for seconds

In [ ]:
import isodate
video_details_df['durationSeconds'] = video_details_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_details_df['durationSeconds'] = video_details_df['durationSeconds'].dt.total_seconds()


Below is for Minutes

In [ ]:
video_details_df[['durationSeconds', 'duration']]

In [ ]:
#getting number of tags
video_details_df['tagCount'] = video_details_df['tags'].apply(lambda x: len(x) if x is not None else 0)

In [ ]:
video_details_df.head(60)

Saving the video details

In [ ]:
video_details_df.to_csv('video_details.csv')

Below I'm just getting a quick visual representation a few stats

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

Highest Viewed Videos

In [ ]:
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_details_df.sort_values('viewCount', ascending = False).head(9))
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/100000) + 'K'))

Lowest viewed

In [ ]:
ax = sns.barplot(x = 'title', y = 'viewCount', data = video_details_df.sort_values('viewCount', ascending = True).head(9))
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/100000) + 'K'))

Violin Plot of Viewr Distribution

In [ ]:
sns.violinplot(x=video_details_df['channelTitle'], y=video_details_df['viewCount'])


Views vs. Likes and Comments

In [ ]:
fig, ax = plt.subplots(1, 2)
sns.scatterplot(data = video_details_df, x = 'commentCount', y = 'viewCount', ax = ax[0])
sns.scatterplot(data = video_details_df, x = 'likeCount', y = 'viewCount', ax = ax[1])

In [ ]:
view_counts = np.random.randint(100000, 10000000, 50)
comment_counts = (view_counts / 100000) + np.random.normal(0, 10, 50)
like_counts = (view_counts / 50) + np.random.normal(0, 1000, 50)

# Create a DF from the data
data = pd.DataFrame({
    'viewCount': view_counts,
    'commentCount': comment_counts,
    'likeCount': like_counts
})

# Pearson correlation coefficients
correlation_view_comment = data['viewCount'].corr(data['commentCount'])
correlation_view_like = data['viewCount'].corr(data['likeCount'])

correlation_view_comment, correlation_view_like

Average Video Length

In [ ]:
sns.histplot(data = video_details_df, x = 'durationSeconds', bins = 50)

In [ ]:
from wordcloud import WordCloud
from wordcloud import WordCloud, STOPWORDS

In [ ]:
stop_words = STOPWORDS
video_details_df['title_no_stopwords'] = video_details_df['title'].apply(lambda x: [item for item in str(x).split() if item not in stop_words])

all_words = list([a for b in video_details_df['title_no_stopwords'].tolist() for a in b])
all_words_str = ' '.join(all_words)

def plot_cloud(wordcloud):
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud) 
    plt.axis("off");

wordcloud = WordCloud(width = 1000, height = 900, random_state=1, background_color='black', colormap='viridis', collocations=False).generate(all_words_str)

plot_cloud(wordcloud)

In [ ]:
day_df = pd.DataFrame(video_details_df['publishDayName'].value_counts())
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_df = day_df.reindex(weekdays)
ax = day_df.reset_index().plot.bar(x = 'publishDayName', y = 'count', rot = 0)

In [ ]:
grouped_comments_df

In [ ]:
comments_df

Open AI Saves to csv

In [ ]:
video_df.to_csv('openai.csv', index = False)

In [ ]:
comments_df.to_csv('openai_comments.csv', index = False)

Kurzgesagt Saves to CSV

In [ ]:
video_details_df.to_csv('kurzgesagt.csv', index = False)

In [ ]:
grouped_comments_df.to_csv('kurzgesagt_comments_original.csv', index = False)

In [ ]:
comments_df.to_csv('kurzgesagt_comments.csv', index = False)